In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
import matplotlib.colors as colors
import glob
import os

# Get all spatial binning files
spatial_binning_files = glob.glob("*_SPATIAL_BINNING_maps_extended.fits")
spatial_binning_files.sort()

print(f"Found {len(spatial_binning_files)} spatial binning files:")
for file in spatial_binning_files:
    print(f"  {file}")

# Extract galaxy names from spatial binning files
galaxy_names = []
for file in spatial_binning_files:
    galaxy_name = file.replace("_SPATIAL_BINNING_maps_extended.fits", "")
    galaxy_names.append(galaxy_name)

print(f"\nGalaxy names: {galaxy_names}")

# Check which gas_BIN files exist
print(f"\nChecking for corresponding gas_BIN files:")
gas_files_exist = []
for galaxy_name in galaxy_names:
    gas_file = f"{galaxy_name}_gas_BIN_maps_extended.fits"
    exists = os.path.exists(gas_file)
    gas_files_exist.append(exists)
    status = "✓" if exists else "✗"
    print(f"  {status} {gas_file}")

# Physical scale parameters
arcsec_per_pixel = 0.2
distance_mpc = 16.5  # You might want to adjust this for each galaxy if distances vary

# Convert arcsec to radians
arcsec_to_rad = np.pi / (180 * 3600)
pixel_size_rad = arcsec_per_pixel * arcsec_to_rad

# Calculate physical size per pixel in kpc
kpc_per_pixel = pixel_size_rad * distance_mpc * 1000

print(f"\nPhysical scale:")
print(f"Pixel size: {arcsec_per_pixel} arcsec")
print(f"Distance: {distance_mpc} Mpc")
print(f"Physical size per pixel: {kpc_per_pixel:.3f} kpc/pixel")

# Counters for tracking processing
processed_count = 0
plotted_count = 0
error_count = 0

# Loop through each galaxy
for i, galaxy_name in enumerate(galaxy_names):
    print(f"\n{'='*60}")
    print(f"Processing galaxy {i+1}/{len(galaxy_names)}: {galaxy_name}")
    print(f"{'='*60}")
    
    # File paths
    spatial_binning_file = f"{galaxy_name}_SPATIAL_BINNING_maps_extended.fits"
    gas_bin_file = f"{galaxy_name}_gas_BIN_maps_extended.fits"
    
    # Check if both files exist
    if not os.path.exists(spatial_binning_file):
        print(f"❌ Warning: {spatial_binning_file} not found, skipping...")
        error_count += 1
        continue
        
    if not os.path.exists(gas_bin_file):
        print(f"❌ Warning: {gas_bin_file} not found, skipping...")
        error_count += 1
        continue
    
    print(f"✓ Both files found for {galaxy_name}")
    
    try:
        # Read the spatial binning file to get LOGMSTAR data and center position
        print(f"📖 Reading spatial binning file...")
        with fits.open(spatial_binning_file) as hdul_spatial:
            # Print available HDUs for debugging
            available_spatial_hdus = [hdu.name for hdu in hdul_spatial]
            print(f"Available HDUs in spatial file: {available_spatial_hdus}")
            
            # Check if LOGMSTAR exists
            if "LOGMSTAR" not in available_spatial_hdus:
                print(f"❌ Warning: LOGMSTAR not found in {spatial_binning_file}, skipping...")
                error_count += 1
                continue
                
            logmstar = hdul_spatial["LOGMSTAR"].data
            print(f"✓ LOGMSTAR shape: {logmstar.shape}")
            print(f"✓ LOGMSTAR - Min: {np.nanmin(logmstar):.3f}, Max: {np.nanmax(logmstar):.3f}")
            
            # Check if all values are NaN
            if np.all(np.isnan(logmstar)):
                print(f"❌ Warning: All LOGMSTAR values are NaN for {galaxy_name}, skipping...")
                error_count += 1
                continue
            
            # Find the position of maximum stellar mass
            max_idx = np.unravel_index(np.nanargmax(logmstar), logmstar.shape)
            center_y, center_x = max_idx
            print(f"✓ Maximum LOGMSTAR position (y, x): ({center_y}, {center_x})")
            print(f"✓ Maximum LOGMSTAR value: {logmstar[center_y, center_x]:.3f}")

        # Read the metallicity maps from the gas binning file
        print(f"📖 Reading gas binning file...")
        with fits.open(gas_bin_file) as hdul:
            # Check if O/H maps exist
            available_gas_hdus = [hdu.name for hdu in hdul]
            print(f"Available HDUs in gas file: {available_gas_hdus}")
            
            if "O_H_D16_SF" not in available_gas_hdus:
                print(f"❌ Warning: O_H_D16_SF not found in {gas_bin_file}, skipping...")
                error_count += 1
                continue
                
            if "O_H_PG16_SF" not in available_gas_hdus:
                print(f"❌ Warning: O_H_PG16_SF not found in {gas_bin_file}, skipping...")
                error_count += 1
                continue
            
            # Read the metallicity maps
            o_h_d16_sf = hdul["O_H_D16_SF"].data
            o_h_pg16_sf = hdul["O_H_PG16_SF"].data
            
            print(f"✓ O_H_D16_SF shape: {o_h_d16_sf.shape}")
            print(f"✓ O_H_PG16_SF shape: {o_h_pg16_sf.shape}")
            
            # Check if all values are NaN
            d16_valid = not np.all(np.isnan(o_h_d16_sf))
            pg16_valid = not np.all(np.isnan(o_h_pg16_sf))
            
            if not d16_valid:
                print(f"❌ Warning: All O_H_D16_SF values are NaN for {galaxy_name}")
            if not pg16_valid:
                print(f"❌ Warning: All O_H_PG16_SF values are NaN for {galaxy_name}")
                
            if not d16_valid or not pg16_valid:
                print(f"❌ Skipping {galaxy_name} due to invalid O/H data")
                error_count += 1
                continue
            
            # Check for data range
            print(f"✓ O_H_D16_SF - Min: {np.nanmin(o_h_d16_sf):.3f}, Max: {np.nanmax(o_h_d16_sf):.3f}")
            print(f"✓ O_H_PG16_SF - Min: {np.nanmin(o_h_pg16_sf):.3f}, Max: {np.nanmax(o_h_pg16_sf):.3f}")

        processed_count += 1
        print(f"✅ Data successfully loaded for {galaxy_name}")

        # Get the shape and create coordinate arrays
        ny, nx = o_h_d16_sf.shape
        print(f"✓ Map dimensions: {ny} × {nx} pixels")

        # Create coordinate arrays centered on the maximum stellar mass position
        x_pixels = np.arange(nx) - center_x
        y_pixels = np.arange(ny) - center_y

        # Convert to physical coordinates in kpc
        x_kpc = x_pixels * kpc_per_pixel
        y_kpc = y_pixels * kpc_per_pixel

        print(f"✓ X coordinate range: {x_kpc[0]:.1f} to {x_kpc[-1]:.1f} kpc")
        print(f"✓ Y coordinate range: {y_kpc[0]:.1f} to {y_kpc[-1]:.1f} kpc")

        # Create the metallicity comparison plots
        print(f"🎨 Creating plots for {galaxy_name}...")
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

        # Plot O_H_D16_SF
        im1 = ax1.imshow(o_h_d16_sf, cmap='inferno', origin='lower', 
                         extent=[x_kpc[0], x_kpc[-1], y_kpc[0], y_kpc[-1]])
        ax1.set_title(f'{galaxy_name}\nO/H D16_SF Method\n[O/H] = 12+log(O/H)', fontsize=12)
        ax1.set_xlabel('Δx (kpc)')
        ax1.set_ylabel('Δy (kpc)')
        cbar1 = plt.colorbar(im1, ax=ax1, shrink=0.8)
        cbar1.set_label('[O/H]', rotation=270, labelpad=15)

        # Plot O_H_PG16_SF
        im2 = ax2.imshow(o_h_pg16_sf, cmap='inferno', origin='lower',
                         extent=[x_kpc[0], x_kpc[-1], y_kpc[0], y_kpc[-1]])
        ax2.set_title(f'{galaxy_name}\nO/H PG16_SF Method\n[O/H] = 12+log(O/H)', fontsize=12)
        ax2.set_xlabel('Δx (kpc)')
        ax2.set_ylabel('Δy (kpc)')
        cbar2 = plt.colorbar(im2, ax=ax2, shrink=0.8)
        cbar2.set_label('[O/H]', rotation=270, labelpad=15)

        plt.tight_layout()
        plt.show()

        # Create a difference map
        fig, ax3 = plt.subplots(1, 1, figsize=(8, 6))
        difference = o_h_d16_sf - o_h_pg16_sf
        
        # Plot the difference map
        im3 = ax3.imshow(difference, cmap='RdBu_r', origin='lower',
                         extent=[x_kpc[0], x_kpc[-1], y_kpc[0], y_kpc[-1]])
        ax3.set_title(f'{galaxy_name}\nDifference: D16_SF - PG16_SF', fontsize=12)
        ax3.set_xlabel('Δx (kpc)')
        ax3.set_ylabel('Δy (kpc)')
        cbar3 = plt.colorbar(im3, ax=ax3, shrink=0.8)
        cbar3.set_label('Δ[O/H]', rotation=270, labelpad=15)
        
        plt.tight_layout()
        plt.show()
        
        plotted_count += 1
        print(f"✅ Plots created for {galaxy_name}")
        
        # Print statistics for the difference
        print(f"📊 Difference statistics:")
        print(f"  Mean difference: {np.nanmean(difference):.4f}")
        print(f"  Std difference: {np.nanstd(difference):.4f}")
        print(f"  Min difference: {np.nanmin(difference):.4f}")
        print(f"  Max difference: {np.nanmax(difference):.4f}")
        
    except Exception as e:
        print(f"❌ Error processing {galaxy_name}: {str(e)}")
        import traceback
        print(f"Full error traceback:")
        traceback.print_exc()
        error_count += 1
        continue

print(f"\n{'='*60}")
print("PROCESSING SUMMARY")
print(f"{'='*60}")
print(f"Total galaxies found: {len(galaxy_names)}")
print(f"Successfully processed: {processed_count}")
print(f"Successfully plotted: {plotted_count}")
print(f"Errors/skipped: {error_count}")
print(f"{'='*60}")